In [3]:
# -*- coding: utf-8 -*-
"""
DLC CSV の likelihood 品質チェック
- 各 CSV について、指定キーポイント群のいずれかで "likelihood <= LIKELIHOOD_THRESH" となるフレームの割合を計算
- その割合が RATIO_THRESHOLD（デフォ 0.5=50%）を超えるファイルを「要修正」として保存
- 出力:
    1) 詳細レポート CSV（各ファイルの総フレーム数、any低信頼フレーム割合、各KPの割合など）
    2) 要修正ファイル名リスト TXT
- DLC 3レベルヘッダ (scorer, bodypart, {x,y,likelihood}) を想定
"""

import os
import re
import glob
import datetime
from pathlib import Path
import numpy as np
import pandas as pd

# ======== 設定（必要に応じて変更）========
# 監視対象ディレクトリ（複数可）: train と eval を一気に確認したい時は配列に追加してください
DIRECTORIES = [
    r"C:\kanno\vscode\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\data\train\train_csv",
    
]

# 使うキーポイント（None なら CSV 内の全キーポイントを対象）
#KEYPOINTS = ["tail_set", "right_tarsal", "right_paw", "left_tarsal", "left_paw"]
KEYPOINTS = None  # ←全キーポイントで評価したい場合はこちら

# likelihood 閾値（この値以下を低信頼と見なす）
LIKELIHOOD_THRESH = 0.6

# 「低信頼フレームの割合（any低信頼）」がこの比率を超えたら要修正と判定（初期値：0.5 = 5割）
RATIO_THRESHOLD = 0.5

# 出力サブフォルダ名（対象ディレクトリの“ひとつ上”に作成します）
OUT_SUBDIR_NAME = "likelihood_qc"

# =========================================


def _norm_name(s: str) -> str:
    """比較用に小文字化 + 空白/アンダーバー/ハイフンを除去"""
    return "".join(ch for ch in s.lower() if ch not in " _-")


def _resolve_keypoints(all_bodyparts, requested):
    """
    CSV内に存在する実名ボディパーツを、requested（正規化済み比較）に応じて抽出。
    requested が None の場合は all_bodyparts（重複除去）をそのまま返す。
    """
    uniq = []
    seen = set()
    for bp in all_bodyparts:
        if bp not in seen:
            uniq.append(bp)
            seen.add(bp)

    if requested is None:
        return uniq

    # 実名 <-> 正規化名 の対応
    norm2orig = {}
    for bp in uniq:
        k = _norm_name(bp)
        if k not in norm2orig:
            norm2orig[k] = bp

    resolved, missing = [], []
    for req in requested:
        k = _norm_name(req)
        if k in norm2orig:
            resolved.append(norm2orig[k])
        else:
            missing.append(req)

    if missing:
        print(f"[WARN] 指定キーポイントがCSVで見つかりません: {missing}\n  利用可能: {uniq}")
    return resolved


def analyze_csv(csv_path: str, use_keypoints):
    """
    単一 CSV を解析し、以下を返す:
      - n_frames: 総フレーム数
      - ratio_any_low: いずれかの対象KPで likelihood<=閾値 となるフレーム割合
      - per-kp の低信頼割合（辞書）
    """
    # 3段ヘッダで読み込み
    df = pd.read_csv(csv_path, header=[0, 1, 2], index_col=0)
    # CSVに存在する全ボディパーツ
    bodyparts = list({bp for (_, bp, _) in df.columns})

    # 対象KPを決める
    use_kps = _resolve_keypoints(bodyparts, use_keypoints)
    if not use_kps:
        raise ValueError(f"対象キーポイントが空です（CSV: {os.path.basename(csv_path)}）")

    # 各KPの likelihood 列を収集
    lik_cols = []
    per_kp_ratio = {}
    for bp in use_kps:
        try:
            lik = df.xs((bp, "likelihood"), level=[1, 2], axis=1).values.flatten()
        except KeyError:
            # likelihood 列が無いKPはスキップ
            print(f"[WARN] likelihood列なし: {bp} in {os.path.basename(csv_path)}")
            continue

        if lik.size == 0:
            continue

        lik_cols.append(lik)
        low_ratio = np.mean(lik <= LIKELIHOOD_THRESH)
        per_kp_ratio[bp] = float(low_ratio)

    if not lik_cols:
        # 対象KPについて likelihood 列が1つもない
        return 0, float("nan"), per_kp_ratio

    # いずれかのKPで低信頼（行方向の OR）
    L = np.column_stack(lik_cols)  # shape (T, K)
    any_low = np.any(L <= LIKELIHOOD_THRESH, axis=1)
    ratio_any_low = float(np.mean(any_low))
    n_frames = int(L.shape[0])

    return n_frames, ratio_any_low, per_kp_ratio


def main():
    date_str = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    all_rows = []
    flagged = []

    for target_dir in DIRECTORIES:
        target_dir = os.path.abspath(target_dir)
        if not os.path.isdir(target_dir):
            print(f"[WARN] ディレクトリなし: {target_dir}")
            continue

        # 出力先は「target_dir のひとつ上」に OUT_SUBDIR_NAME をぶら下げる
        parent = os.path.dirname(target_dir)
        out_dir = os.path.join(parent, OUT_SUBDIR_NAME)
        os.makedirs(out_dir, exist_ok=True)

        print(f"[INFO] 走査: {target_dir}")
        csv_list = sorted(glob.glob(os.path.join(target_dir, "*.csv")))
        if not csv_list:
            print(f"[WARN] CSV が見つかりません: {target_dir}")
            continue

        for csv_path in csv_list:
            try:
                n_frames, ratio_any_low, per_kp_ratio = analyze_csv(csv_path, KEYPOINTS)
            except Exception as e:
                print(f"[ERROR] 解析失敗: {os.path.basename(csv_path)} -> {e}")
                continue

            row = {
                "dir": target_dir,
                "file": os.path.basename(csv_path),
                "n_frames": n_frames,
                "ratio_any_low": ratio_any_low,
                "threshold_likelihood": LIKELIHOOD_THRESH,
                "threshold_ratio": RATIO_THRESHOLD,
            }
            # KPごとの割合も列として付与（列名: ratio_<normed_kpname>）
            for bp, r in per_kp_ratio.items():
                col = "ratio_" + _norm_name(bp)
                row[col] = r

            all_rows.append(row)

            # 閾値超過ファイルをフラグ
            if n_frames > 0 and ratio_any_low > RATIO_THRESHOLD:
                flagged.append(os.path.join(target_dir, os.path.basename(csv_path)))

        # 各 target_dir ごとに保存（タイムスタンプ付き）
        if all_rows:
            df = pd.DataFrame(all_rows)
            # 同じ parent のものだけ抜き出して保存（見やすさのため）
            df_parent = df[df["dir"] == target_dir].copy()

            csv_out = os.path.join(out_dir, f"dlc_lowlikelihood_report_{date_str}.csv")
            df_parent.to_csv(csv_out, index=False, encoding="utf-8-sig")
            print(f"[INFO] レポート保存: {csv_out}")

            # フラグ対象のみの TXT
            flagged_parent = [p for p in flagged if os.path.dirname(p) == target_dir]
            txt_out = os.path.join(out_dir, f"dlc_lowlikelihood_problem_files_{date_str}.txt")
            with open(txt_out, "w", encoding="utf-8") as f:
                for p in flagged_parent:
                    f.write(p + "\n")
            print(f"[INFO] 要修正ファイル一覧: {txt_out}")

    # 全体サマリ
    print("\n===== SUMMARY =====")
    print(f"対象ディレクトリ数: {len(DIRECTORIES)}")
    print(f"解析ファイル総数:   {len(set(os.path.basename(p) for p in flagged)) + 0 if not all_rows else sum(1 for _ in all_rows)}")
    print(f"要修正ファイル数:   {len(flagged)}")
    if flagged:
        print("例:")
        for p in flagged[:10]:
            print(" -", p)


if __name__ == "__main__":
    main()


[INFO] 走査: C:\kanno\vscode\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\data\train\train_csv
[INFO] レポート保存: C:\kanno\vscode\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\data\train\likelihood_qc\dlc_lowlikelihood_report_20251219-154455.csv
[INFO] 要修正ファイル一覧: C:\kanno\vscode\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\data\train\likelihood_qc\dlc_lowlikelihood_problem_files_20251219-154455.txt

===== SUMMARY =====
対象ディレクトリ数: 1
解析ファイル総数:   117
要修正ファイル数:   106
例:
 - C:\kanno\vscode\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\data\train\train_csv\normal_10DLC_resnet50_sotuken1Dec17shuffle1_100000.csv
 - C:\kanno\vscode\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master